<a href="https://colab.research.google.com/github/Pjh940114/ch_project_3/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC(%EB%84%A4%EC%9D%B4%EB%B2%84%ED%86%A0%EB%A1%A0%EB%B0%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datetime import datetime
import re
import os
import numpy as np
import time

In [ ]:
# 경로 및 기본설정
start = time.time()

default_path = '/content/drive/MyDrive/krx_nlp/'
default_list = ['lg_chem', 'samsung_sdi', 'sk_innov', 'posco_chem', 'korea_zinc'] # 네이버 다음 주식 이름 통일

naver_raw_list = ['LG화학', '삼성SDI', 'SK이노베이션', '포스코케미칼', '고려아연']
daum_raw_list = ['LGCM', 'SSSDI', 'SKIV', 'POSCOCM', 'KoreaZinc']

naver_to_default = dict(zip(naver_raw_list, default_list))
daum_to_default = dict(zip(daum_raw_list, default_list))

naver = 'naver_board_'
daum = 'daum_talks_'

# stopwords 설정
stopwords = ['삭제된 게시물의 답글', '[자막뉴스]']

# 댓글 달린 수 삭제
for i in range(20):
  stopwords.append(str([i]))

In [ ]:
# 파일 이름 포맷 통일
def changefilename(path):
  for k, v in naver_to_default.items():
    os.rename(path + naver + k + '.csv', path + v + '_naver' + '.csv')

  for k, v in daum_to_default.items():
    os.rename(path + daum + k + '.csv', path + v + '_daum' + '.csv')

################ 최초 1회 실행 #################
# changefilename(default_path + 'rawdata/')

In [ ]:
# 불러오기
def read_csv(files, path):
  for file in files:
    try:
      globals()[file.split('.')[0]] = pd.read_csv(path + '{}'.format(file), encoding = 'utf-8-sig')
    except:
      print("not read :", file)

read_csv(os.listdir(default_path + 'rawdata/'), default_path + 'rawdata/')

not read : sample_stock_data.csv
not read : .ipynb_checkpoints


In [ ]:
def set_df(channel, df):
  print("전처리 전 데이터 길이 :", len(df))
  # df 역순으로 만들기
  r_idx = [i for i in range(df.shape[0]-1, -1, -1)]
  r_df = pd.DataFrame(df, index = r_idx)

  # 인덱스 초기화
  r_df.reset_index(drop = True, inplace = True)
  df = r_df

  # column 제거 및 이름 변경
  if channel == 'naver':
    df.drop(columns = ['글쓴이', '조회', '공감', '비공감'], inplace = True)

  elif channel == 'daum':
    df.drop(columns = ['Unnamed: 0', '종목명'], inplace = True)

    # column 순서 변경
    col1 = df.columns[1:].to_list()
    col2 = df.columns[:1].to_list()
    new_col = col1 + col2
    df = df[new_col]

  # column명 통일
  df.columns = ['day', 'title']

  # 날짜 세팅(YYYY-MM-DD)
  df['day'] = df['day'].str.replace("[^0-9]", "")

  if channel == 'naver':
    for i in range(len(df['day'])):
      df['day'][i] = df['day'][i][:8]

    # for i in range(len(df)):
    #   df.day[i] = datetime.strptime(str(df.day[i]), '%Y%m%d')

  # if channel == 'daum':
  #   for i in range(len(df)):
  #     df.day[i] = datetime.strptime(str(df.day[i]), '%y%m%d')

  # 날짜에서 시간 제거
  df['day'] = df['day'].astype(str).str.replace("00:00:00","")

  # 전처리

  # 특수문자 제거 : 한글, 영어, 숫자 살리고 특수문자 -> replace : ex) +/-/%/↑/↓/...
  df['title'] = df['title'].str.replace('[^一-龥ㄱ-힣a-zA-Z0-9\s]+',"",regex=True)
  # df['Title'] = df['Title'].str.replace('[^가-힣\sㄱ-ㅎ0-9+-]+',"",regex=True)
  # df['Title'] = df['Title'].str.replace('^[ㄱ-ㅎ|가-힣|a-z|A-Z|0-9|]*$',"",regex=True)

  # 공백 제거
  df['title'] = df['title'].replace('', np.nan) # 자음까지 살림
  df = df.dropna(how = 'any')

  # stopwords 제거
  df['title'] = df['title'].str.replace(f"[{stopwords}]","")

  # 제목 중복 제거
  df.drop_duplicates(subset = ['title'], inplace = True)

  print("전처리 후 데이터 길이 :", len(df))
  print(f"{time.time() - start:.5f} sec")
  return df

In [ ]:
lg_chem_naver = set_df('naver', lg_chem_naver)
lg_chem_daum = set_df('daum', lg_chem_daum)
samsung_sdi_naver = set_df('naver', samsung_sdi_naver)
samsung_sdi_daum = set_df('daum', samsung_sdi_daum)
sk_innov_naver = set_df('naver', sk_innov_naver)
sk_innov_daum = set_df('daum', sk_innov_daum)
posco_chem_naver = set_df('naver', posco_chem_naver)
posco_chem_daum = set_df('daum', posco_chem_daum)
korea_zinc_naver = set_df('naver', korea_zinc_naver)
korea_zinc_daum = set_df('daum', korea_zinc_daum)

전처리 전 데이터 길이 : 66620


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwar

전처리 후 데이터 길이 : 57591
26.56341 sec
전처리 전 데이터 길이 : 573
전처리 후 데이터 길이 : 473
26.57944 sec
전처리 전 데이터 길이 : 44084
전처리 후 데이터 길이 : 37370
37.76559 sec
전처리 전 데이터 길이 : 313
전처리 후 데이터 길이 : 269
37.77949 sec
전처리 전 데이터 길이 : 101211
전처리 후 데이터 길이 : 87654
90.71582 sec
전처리 전 데이터 길이 : 288
전처리 후 데이터 길이 : 253
90.73063 sec
전처리 전 데이터 길이 : 33161
전처리 후 데이터 길이 : 28705
100.01965 sec
전처리 전 데이터 길이 : 120
전처리 후 데이터 길이 : 114
100.02928 sec
전처리 전 데이터 길이 : 4303
전처리 후 데이터 길이 : 3940
100.41996 sec
전처리 전 데이터 길이 : 44
전처리 후 데이터 길이 : 42
100.42953 sec


In [ ]:
# 기간 설정
start_date = '2021-01-01'
end_date = '2022-06-30'

def join_date(df, start_date, end_date):
  mask = (df['Date'] > start_date) & (df['Date'] <= end_date)
  filtered_df = df.loc[mask]

  return filtered_df

In [ ]:
lg_chem_naver = join_date(lg_chem_naver, start_date, end_date)
lg_chem_daum = join_date(lg_chem_daum, start_date, end_date)

samsung_sdi_naver = join_date(samsung_sdi_naver, start_date, end_date)
samsung_sdi_daum = join_date(samsung_sdi_daum, start_date, end_date)

sk_innov_naver = join_date(sk_innov_naver, start_date, end_date)
sk_innov_daum = join_date(sk_innov_daum, start_date, end_date)

posco_chem_naver = join_date(posco_chem_naver, start_date, end_date)
posco_chem_daum = join_date(posco_chem_daum, start_date, end_date)

korea_zinc_naver = join_date(korea_zinc_naver, start_date, end_date)
korea_zinc_daum = join_date( korea_zinc_daum, start_date, end_date)

In [ ]:
# csv 로 저장하기
def save_csv(title, df, start_date, end_date):
  filename = title + "_v1.csv"
  df.to_csv(default_path + 'predata/' + start_date + '_to_' + end_date + '_' +filename, mode='w', encoding = 'utf-8-sig')
  print("saved file!!\nfilename : {}".format(filename))

In [ ]:
save_csv("lg_chme_naver", lg_chem_naver, start_date, end_date)
save_csv("lg_chme_daum", lg_chem_daum, start_date, end_date)

save_csv("samsung_sdi_naver", samsung_sdi_naver, start_date, end_date)
save_csv("samsung_sdi_daum", samsung_sdi_daum, start_date, end_date)

save_csv("sk_innov_naver", sk_innov_naver, start_date, end_date)
save_csv("sk_innov_daum", sk_innov_daum, start_date, end_date)

save_csv("posco_chem_naver", posco_chem_naver, start_date, end_date)
save_csv("posco_chem_daum", posco_chem_daum, start_date, end_date)

save_csv("korea_zinc_naver", korea_zinc_naver, start_date, end_date)
save_csv("korea_zinc_daum", korea_zinc_daum, start_date, end_date)

saved file!!
filename : lg_chme_naver_v1.csv
saved file!!
filename : lg_chme_daum_v1.csv
saved file!!
filename : samsung_sdi_naver_v1.csv
saved file!!
filename : samsung_sdi_daum_v1.csv
saved file!!
filename : sk_innov_naver_v1.csv
saved file!!
filename : sk_innov_daum_v1.csv
saved file!!
filename : posco_chem_naver_v1.csv
saved file!!
filename : posco_chem_daum_v1.csv
saved file!!
filename : korea_zinc_naver_v1.csv
saved file!!
filename : korea_zinc_daum_v1.csv


In [ ]:
# 네이버 + 다음 종토방 데이터 합치기
def concat_df(naver_df, daum_df):
  df = pd.concat([naver_df, daum_df])
  return df